In [1]:
import numpy as np
import plotly.graph_objects as go
import random

In [2]:
def obj_data_to_mesh3d(odata):
    """
    Convert OBJ file data to vertices and faces for 3D mesh rendering.
    
    Args:
        odata (str): Raw text data from an OBJ file
    
    Returns:
        tuple: Arrays of vertices and faces
    """
    vertices = []
    faces = []
    lines = odata.splitlines()   
   
    for line in lines:
        slist = line.split()
        if slist:
            if slist[0] == 'v':
                vertex = np.array(slist[1:], dtype=float)
                vertices.append(vertex)
            elif slist[0] == 'f':
                face = []
                for k in range(1, len(slist)):
                    face.append([int(s) for s in slist[k].replace('//','/').split('/')])
                if len(face) > 3:  # triangulate the n-polyonal face, n>3
                    faces.extend([[face[0][0]-1, face[k][0]-1, face[k+1][0]-1] for k in range(1, len(face)-1)])
                else:    
                    faces.append([face[j][0]-1 for j in range(len(face))])
    
    return np.array(vertices), np.array(faces)

In [3]:
def generate_raffle_participants(num_participants=50):
    """
    Generate a list of dummy raffle participants.
    
    Args:
        num_participants (int): Number of participants to generate
    
    Returns:
        list: List of dictionaries with participant details
    """
    first_names = [
        "Emma", "Liam", "Olivia", "Noah", "Ava", "Ethan", "Sophia", "Mason", 
        "Isabella", "William", "Mia", "James", "Charlotte", "Benjamin", "Amelia", 
        "Lucas", "Harper", "Henry", "Evelyn", "Alexander"
    ]
    last_names = [
        "Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia", "Miller", 
        "Davis", "Rodriguez", "Martinez", "Hernandez", "Lopez", "Gonzalez", 
        "Wilson", "Anderson", "Taylor", "Moore", "Jackson", "Martin", "Lee"
    ]
    
    participants = []
    for _ in range(num_participants):
        first = random.choice(first_names)
        last = random.choice(last_names)
        email = f"{first.lower()}.{last.lower()}@example.com"
        participants.append({
            "name": f"{first} {last}",
            "email": email
        })
    
    return participants

In [11]:
def generate_participant_points(vertices, participants):
    """
    Generate 3D points for raffle participants within the brain mesh.
    
    Args:
        vertices (np.ndarray): Array of mesh vertices
        participants (list): List of participant dictionaries
    
    Returns:
        tuple: (points, participants_with_points)
    """
    # Calculate bounding box
    x_min, x_max = np.min(vertices[:, 0]), np.max(vertices[:, 0])
    y_min, y_max = np.min(vertices[:, 1]), np.max(vertices[:, 1])
    z_min, z_max = np.min(vertices[:, 2]), np.max(vertices[:, 2])
    
    # Generate random points within the bounding box
    points = np.column_stack([
        np.random.uniform(x_min, x_max, len(participants)),
        np.random.uniform(y_min, y_max, len(participants)),
        np.random.uniform(z_min, z_max, len(participants))
    ])
    
    # Add points to participants
    for i, participant in enumerate(participants):
        participant['point'] = points[i]
    
    return points, participants

In [13]:
def generate_bisection_plane(vertices):
    """
    Generate a random plane that bisects the 3D space.
    
    Args:
        vertices (np.ndarray): Array of mesh vertices
    
    Returns:
        tuple: (plane_point, plane_normal)
    """
    # Calculate center of the mesh
    center = np.mean(vertices, axis=0)
    
    # Generate a random normal vector
    plane_normal = np.random.randn(3)
    plane_normal = plane_normal / np.linalg.norm(plane_normal)
    
    return center, plane_normal

In [4]:
def point_plane_distance(point, plane_point, plane_normal):
    """
    Calculate the signed distance of a point from a plane.
    
    Args:
        point (np.ndarray): 3D point coordinates
        plane_point (np.ndarray): Point on the plane
        plane_normal (np.ndarray): Normal vector of the plane
    
    Returns:
        float: Signed distance from the point to the plane
    """
    return np.dot(point - plane_point, plane_normal)

In [ ]:
def create_brain_raffle_visualization(obj_file, pre_draw=False, r_participants=None):
    """
    Create a 3D brain mesh raffle draw visualization.
    
    Args:
        obj_file (str): Path to brain surface OBJ file
        pre_draw (bool): If True then only the brain and participants points are plotted
        r_participants (None | dict): If None then dummy participants are made. If you
            have particpants they must be formatted in a list of dicts with keys 'email' and
            'name'.
    
    Returns:
        tuple: (plotly figure, winner details)
    """
    # Load OBJ file
    with open(obj_file, "r") as f:
        obj_data = f.read()
    
    # Extract vertices and faces
    vertices, faces = obj_data_to_mesh3d(obj_data)
    vert_x, vert_y, vert_z = vertices[:,:3].T
    face_i, face_j, face_k = faces.T

    # Generate participants and their points
    if r_participants is not None:
        participants = r_participants
    else:
        participants = generate_raffle_participants()
        
    points, participants_with_points = generate_participant_points(vertices, participants)

    # Generate bisection plane
    plane_point, plane_normal = generate_bisection_plane(vertices)

    # Find the winner (point closest to the plane)
    distances = [abs(point_plane_distance(p['point'], plane_point, plane_normal)) for p in participants_with_points]
    winner_index = np.argmin(distances)
    winner = participants_with_points[winner_index]
    winner_distance = distances[winner_index]

    # Create figure
    fig = go.Figure()
    
    # Update layout
    fig.update_layout(
        scene=dict(
            xaxis=dict(showticklabels=False, visible=False),
            yaxis=dict(showticklabels=False, visible=False),
            zaxis=dict(showticklabels=False, visible=False),
        ),
        width=800, 
        height=600,
        title='3D Brain Mesh Raffle Draw'
    )
    
    if pre_draw is True:
        
        # Add brain surface mesh
        fig.add_trace(go.Mesh3d(
            x=vert_x, y=vert_y, z=vert_z, 
            i=face_i, j=face_j, k=face_k,
            color='#ef619f', 
            opacity=0.4, 
            name='Brain Surface', 
            showscale=False, 
            hoverinfo='none'
        ))

        # Add participant points
        fig.add_trace(go.Scatter3d(
            x=points[:, 0],
            y=points[:, 1],
            z=points[:, 2],
            mode='markers',
            marker=dict(
                size=5,
                color='#3abff0',
                opacity=0.7
            ),
            name='Participants'
        ))
        
        return fig, _, _

    # Add brain surface mesh
    fig.add_trace(go.Mesh3d(
        x=vert_x, y=vert_y, z=vert_z, 
        i=face_i, j=face_j, k=face_k,
        color='#ef619f', 
        opacity=0.4, 
        name='Brain Surface', 
        showscale=False, 
        hoverinfo='none'
    ))

    # Add participant points
    fig.add_trace(go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(
            size=5,
            color='#3abff0',
            opacity=0.7
        ),
        name='Participants'
    ))

    # Add winner point
    fig.add_trace(go.Scatter3d(
        x=[winner['point'][0]],
        y=[winner['point'][1]],
        z=[winner['point'][2]],
        mode='markers',
        marker=dict(
            size=10,
            color='gold',
            opacity=1
        ),
        name='Winner'
    ))

    # Add plane visualization
    # Create a plane with points spanning the mesh
    d = -np.dot(plane_point, plane_normal)
    xx, yy = np.meshgrid(
        np.linspace(np.min(vert_x), np.max(vert_x), 10),
        np.linspace(np.min(vert_y), np.max(vert_y), 10)
    )
    z = (-plane_normal[0] * xx - plane_normal[1] * yy - d) * 1.0 / plane_normal[2]

    fig.add_trace(go.Surface(
        x=xx, y=yy, z=z,
        opacity=0.5,
        colorscale=[[0, 'green'], [1, 'green']],
        showscale=False
    ))

    # Update layout
    fig.update_layout(
        scene=dict(
            xaxis=dict(showticklabels=False, visible=False),
            yaxis=dict(showticklabels=False, visible=False),
            zaxis=dict(showticklabels=False, visible=False),
        ),
        width=800, 
        height=600,
        title='3D Brain Mesh Raffle Draw'
    )

    return fig, winner, winner_distance



In [6]:
import pandas as pd

In [ ]:
# How i augmented our CSV of participants data into the necessary format
sign_ups = pd.read_csv('../100Biologists_responses.csv')
from_here = sign_ups[sign_ups['Column 1'] == 'RAFFLE FROM  HERE DOWN'].index[0]
raffle = sign_ups.iloc[from_here:,1:3]
raffle.rename({'Name:': 'name', 'Email address': 'email'}, axis='columns', inplace=True)
raffle_dict = raffle.to_dict(orient='records')
# raffle_dict

In [ ]:
# Show the participants data points in space, pre raffle draw
fig, _, _ = create_brain_raffle_visualization(obj_file="../lh.pial.obj", 
                                              pre_draw=True,
                                              r_participants=None)
fig.show()

In [ ]:
# Run the raffle!
fig, winner, distance = create_brain_raffle_visualization(obj_file="../lh.pial.obj", 
                                              pre_draw=False,
                                              r_participants=raffle_dict)
fig.show()

print(f"Winner: {winner['name']} (Email: {winner['email']})")
print(f"Distance from plane: {distance}")